In [387]:
!pip install langchain_community
!pip install langchain_text_splitter
!pip install langchain_openai
!pip install langchain_chroma
!pip install transformers
# !pip install python-dotenv

ERROR: Could not find a version that satisfies the requirement langchain_text_splitter (from versions: none)
ERROR: No matching distribution found for langchain_text_splitter


In [388]:
from langchain_community.document_loaders import TextLoader # ubah teks ke format langchain
from langchain_text_splitters import CharacterTextSplitter # Split text
from langchain_openai import OpenAIEmbeddings # convert ke document embedding
from langchain_chroma import Chroma # vector database
import pandas as pd
from google.colab import userdata
from transformers import pipeline
import numpy as np
from tqdm import tqdm

OPEN_KEY = userdata.get('OPENAI_KEY')
HF_KEY = userdata.get('HF_KEY')

embedding = OpenAIEmbeddings(openai_api_key=OPEN_KEY)

## Semantic Search

In [389]:
books = pd.read_csv('book_cleaned.csv')
books.head(3)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title & subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."


Penjelasan mengenai _tagged decription_:

> Saat membuat vector search dan mengkueri databasenya, yang bakal didapetin itu deskripsi bukunya karna itulah yang ada di VD (vector database). Tapi kita tidak mungkin ngasih rekomendasi yang isinya deskripsi aja ke user, karna tidak masuk akal kalau cuma ngasih deskripsi tapi gaada judul ataupun authors.

> Yang harus dilakukan adalah filter data frame yang berisi semua buku yang direkomendasikan bedasarkan kueri VD. Kalau kita lakukan ini pakai _string match_ bakal jadi berantakan. Yang mana pelan dan tidak presisi.

> Jadi yang dilakukan adalah menggunakan ISBN sebagai identifier. Yang nantinya ketika dapat rekomendasinya, ISBN nya bakal di _split_ dari deskripsi yang mana ini bakal jadi filternya

>Note: _Text Loader_ di langchain tidak bisa menggunakan dataframe dari pandas

In [390]:
# Hanya sekali run

books["tagged_description"].to_csv("tagged_description.txt",
                                   sep="\n",
                                   index=False,
                                   header=False
                                   )

In [391]:
from google.colab import files
files.download('tagged_description.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [392]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=0,
                                      chunk_overlap=0,
                                      separator="\n"
                                      )
documents = text_splitter.split_documents(raw_documents)

Streaming output truncated to the last 5000 lines.


In [393]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [394]:
db_books = Chroma.from_documents(
    documents,
    embedding = embedding,
    persist_directory="db_chroma"  # Direktori untuk menyimpan database
)

In [395]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
print(docs) # Hasilnya cuma deskripsi aja

[Document(id='a94ff0a0-335d-4192-8feb-e173cace64ac', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'), Document(id='c0455e36-9780-4a25-930c-f6d549bb34be', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'), Document(id='7aabbf20-1229-428e-a3b2-56cbb53d3c69', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction

In [396]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title & subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [397]:
def fetch_rekomendasi_semantic(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k= 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]
    return books[books["isbn13"].isin(books_list)].head(top_k)

In [398]:
fetch_rekomendasi_semantic("A book to teach learn about history")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title & subtitle,tagged_description
575,9780140153194,0140153195,Justine,Lawrence Durrell,Fiction,http://books.google.com/books/content?id=zuK-P...,On the eve of World War II in the Egyptian cit...,1991.0,3.90,253.0,5183.0,Justine,9780140153194 On the eve of World War II in th...
698,9780140449082,0140449086,The Histories,Herodotus,History,http://books.google.com/books/content?id=x-gAo...,One of the masterpieces of classical literatur...,2003.0,3.98,716.0,32923.0,The Histories,9780140449082 One of the masterpieces of class...
957,9780192824257,0192824252,The Histories,Herodotus,History,http://books.google.com/books/content?id=VrV5T...,Provides a new translation of the classic acco...,1998.0,3.98,772.0,265.0,The Histories,9780192824257 Provides a new translation of th...
1533,9780345468031,0345468031,The Franklin Affair,Jim Lehrer,Fiction,http://books.google.com/books/content?id=dhGZP...,"Following the death of his longtime mentor, a ...",2006.0,3.12,208.0,141.0,The Franklin Affair,9780345468031 Following the death of his longt...
2112,9780439405577,0439405572,Don't Know Much about American History,Kenneth C. Davis,United States,NaN,"Presents, in question and answer format, a his...",2003.0,3.90,222.0,21169.0,Don't Know Much about American History,"9780439405577 Presents, in question and answer..."
2418,9780451528308,0451528301,Ten Great Works of Philosophy,Robert Paul Wolff,Fiction,http://books.google.com/books/content?id=uMxnF...,"From ancient Greece to 19th-century America, t...",2002.0,3.82,592.0,85.0,Ten Great Works of Philosophy,9780451528308 From ancient Greece to 19th-cent...
2459,9780465024964,0465024963,The Classical World,Robin Lane Fox,History,http://books.google.com/books/content?id=Srhcj...,"Armies and empires, statesmen and tyrants--the...",2005.0,3.92,672.0,1222.0,The Classical World: An Epic History from Home...,"9780465024964 Armies and empires, statesmen an..."
4802,9781586631987,1586631985,The American Heritage New History of the Civil...,Bruce Catton,History,http://books.google.com/books/content?id=7SddA...,A history of the Civil War that features over ...,2001.0,4.10,630.0,100.0,The American Heritage New History of the Civil...,9781586631987 A history of the Civil War that ...


## Klasifikasi Teks

In [399]:
books["categories"].value_counts()

,count
categories,
Fiction,2111
Juvenile Fiction,390
Biography & Autobiography,311
History,207
Literary Criticism,124
...,...
Human-animal relationships,1
Imperialism,1
Aged women,1


> Kategorinya terlalu bervariasi jadi bakal dijadikan lebih sedikit

In [400]:
books["categories"].value_counts().reset_index().query("count > 50")

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Religion,117
6,Philosophy,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [401]:
mapping_kategori = {
    "Fiction":"Fiction",
    "Juvenile Fiction":"Children's Fiction",
    "Biography & Autobiography":"Nonfiction",
    "History":"Nonfiction",
    "Literary Criticism":"Nonfiction",
    "Religion":"Nonfiction",
    "Philosophy":"Nonfiction",
   "Comics & Graphic Novels":"Fiction" ,
    "Drama":"Fiction",
    "Juvenile Nonfiction":"Children's Nonfiction",
    "Science":"Nonfiction",
    "Poetry":"Fiction"

}

books['simple_categories'] = books['categories'].map(mapping_kategori)

In [402]:
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title & subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,NaN
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,NaN
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",NaN


In [403]:
books[~(books['simple_categories'].isna())].shape

(3743, 14)

In [404]:
pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device="cuda")

Device set to use cuda


In [405]:
sequence = books.loc[books['simple_categories'] == "Fiction", "description"].reset_index(drop=True)[0]

In [406]:
pipe(sequence, fiction_kategori)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [407]:
skor_max = np.argmax(pipe(sequence, fiction_kategori)["scores"])
label_max = pipe(sequence, fiction_kategori)["labels"][skor_max]

label_max

'Fiction'

In [408]:
kategori_fiks = ['Fiction', 'Nonfiction']

In [409]:
def prediksi(sequence, categories):
  prediksi = pipe(sequence, categories)
  skor_max = np.argmax(prediksi["scores"])
  label_max = prediksi["labels"][skor_max]

  return label_max

> Pertama akan coba pre-trained model di 300 data pertama

In [410]:
kategori_asli = []
kategori_prediksi = []

for i in tqdm(range(0,300)):
  sequence = books.loc[books['simple_categories'] == "Fiction", "description"].reset_index(drop=True)[i]
  kategori_prediksi += [prediksi(sequence, kategori_fiks)]
  kategori_asli += ['Fiction']

100%|██████████| 300/300 [00:13<00:00, 21.56it/s]


In [411]:
for i in tqdm(range(0,300)):
  sequence = books.loc[books['simple_categories'] == "Nonfiction", "description"].reset_index(drop=True)[i]
  kategori_prediksi += [prediksi(sequence, kategori_fiks)]
  kategori_asli += ['Nonfiction']

100%|██████████| 300/300 [00:14<00:00, 20.60it/s]


In [412]:
df_prediksi = pd.DataFrame({
    "asli":kategori_asli,
    "prediksi":kategori_prediksi
})
df_prediksi.head() # Ini perlu dibandingkan lagi

,asli,prediksi
0,Fiction,Fiction
1,Fiction,Fiction
2,Fiction,Fiction
3,Fiction,Nonfiction
4,Fiction,Fiction


In [413]:
df_prediksi["prediksi_benar"] = (
    np.where(df_prediksi["asli"] == df_prediksi["prediksi"], 1, 0)
)
df_prediksi['prediksi_benar'].sum()/len(df_prediksi)

np.float64(0.7783333333333333)

77 % benar, berarti bisa digunakan Skarang kita pakai buat missing data

In [414]:
isbn = []
prediksi_kategori = []

missing_kate = books.loc[books["simple_categories"].isna(), ["isbn13","description"]].reset_index(drop=True)

In [415]:
for i in tqdm(range(0, len(missing_kate))):
  isbn += [missing_kate.loc[i, "isbn13"]]
  prediksi_kategori += [prediksi(missing_kate.loc[i, "description"], kategori_fiks)]

100%|██████████| 1454/1454 [01:05<00:00, 22.31it/s]


In [416]:
missing_kate_df = pd.DataFrame({
    "isbn13":isbn,
    "kategori":prediksi_kategori
})
missing_kate_df.head()

,isbn13,kategori
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction


> Merge dataset kategori ke dataset

In [417]:
books= pd.merge(books, missing_kate_df, on="isbn13", how="left")
books["simple_categories"] = np.where(books["simple_categories"].isna(), books["kategori"], books["simple_categories"])

In [418]:
books = books.drop(columns= ['kategori'])

In [419]:
books.to_csv('categorized_books.csv',index=False)

In [420]:
from google.colab import files
files.download('categorized_books.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [421]:
sorted(books["simple_categories"].unique())

["Children's Fiction", "Children's Nonfiction", 'Fiction', 'Nonfiction']

## Sentiment Analysis

In [422]:
new_books = pd.read_csv('categorized_books.csv') # tes
new_books.head(3)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title & subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction


In [423]:
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      return_all_scores=True,
                      top_k = None,
                      device="cuda"
                      )
classifier("I Love this book!") # Tes model

Device set to use cuda


[[{'label': 'joy', 'score': 0.976298451423645},
  {'label': 'surprise', 'score': 0.012351829558610916},
  {'label': 'neutral', 'score': 0.0049095177091658115},
  {'label': 'sadness', 'score': 0.002730812644585967},
  {'label': 'anger', 'score': 0.002324493834748864},
  {'label': 'disgust', 'score': 0.000938638171646744},
  {'label': 'fear', 'score': 0.00044629830517806113}]]

In [424]:
new_books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [425]:
classifier(new_books['description'][0])

[[{'label': 'fear', 'score': 0.6548417210578918},
  {'label': 'neutral', 'score': 0.169851616024971},
  {'label': 'sadness', 'score': 0.1164090484380722},
  {'label': 'surprise', 'score': 0.020700674504041672},
  {'label': 'disgust', 'score': 0.01910070702433586},
  {'label': 'joy', 'score': 0.015161193907260895},
  {'label': 'anger', 'score': 0.003935156390070915}]]

> Kalau langsung masukin deskripsinya dapat 65% fear, yang sebenarnya salah. Gimana kalau split per kalimat?

In [426]:
kalimat = new_books['description'][0].split(".")
prediksi = classifier(kalimat)

In [427]:
kalimat[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [428]:
prediksi[0]

[{'label': 'surprise', 'score': 0.7296027541160583},
 {'label': 'neutral', 'score': 0.14038559794425964},
 {'label': 'fear', 'score': 0.06816219538450241},
 {'label': 'joy', 'score': 0.04794243723154068},
 {'label': 'anger', 'score': 0.009156357496976852},
 {'label': 'disgust', 'score': 0.0026284728664904833},
 {'label': 'sadness', 'score': 0.002122161677107215}]

In [429]:
kalimat[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [430]:
prediksi[3]

[{'label': 'fear', 'score': 0.9281679391860962},
 {'label': 'anger', 'score': 0.03219100460410118},
 {'label': 'neutral', 'score': 0.01280873455107212},
 {'label': 'sadness', 'score': 0.008756879717111588},
 {'label': 'surprise', 'score': 0.008597937412559986},
 {'label': 'disgust', 'score': 0.008431851863861084},
 {'label': 'joy', 'score': 0.0010455828160047531}]

> Classifiernya aman

In [431]:
prediksi

[[{'label': 'surprise', 'score': 0.7296027541160583},
  {'label': 'neutral', 'score': 0.14038559794425964},
  {'label': 'fear', 'score': 0.06816219538450241},
  {'label': 'joy', 'score': 0.04794243723154068},
  {'label': 'anger', 'score': 0.009156357496976852},
  {'label': 'disgust', 'score': 0.0026284728664904833},
  {'label': 'sadness', 'score': 0.002122161677107215}],
 [{'label': 'neutral', 'score': 0.44937053322792053},
  {'label': 'disgust', 'score': 0.2735919654369354},
  {'label': 'joy', 'score': 0.10908277332782745},
  {'label': 'sadness', 'score': 0.09362724423408508},
  {'label': 'anger', 'score': 0.04047832265496254},
  {'label': 'surprise', 'score': 0.02697017788887024},
  {'label': 'fear', 'score': 0.006879044696688652}],
 [{'label': 'neutral', 'score': 0.6462171077728271},
  {'label': 'sadness', 'score': 0.24273230135440826},
  {'label': 'disgust', 'score': 0.04342260956764221},
  {'label': 'surprise', 'score': 0.028300557285547256},
  {'label': 'joy', 'score': 0.01421146

In [432]:
sorted(prediksi[0], key=lambda x: x['label'])

[{'label': 'anger', 'score': 0.009156357496976852},
 {'label': 'disgust', 'score': 0.0026284728664904833},
 {'label': 'fear', 'score': 0.06816219538450241},
 {'label': 'joy', 'score': 0.04794243723154068},
 {'label': 'neutral', 'score': 0.14038559794425964},
 {'label': 'sadness', 'score': 0.002122161677107215},
 {'label': 'surprise', 'score': 0.7296027541160583}]

> Nah sekarang kita coba sentimen analisis di 5 buku untuk semua emotion yang ada

In [433]:
label_emosi = ["anger","disgust","fear","joy","sadness","surprise","neutral"]
isbn = []


def hitung_skor_emosi(prediksi):
  per_emosi = {label: [] for label in label_emosi}

  for predik in prediksi:
    sorted_prediksi = sorted(predik, key=lambda x: x['label'])
    for index, label in enumerate(label_emosi):
      per_emosi[label].append(sorted_prediksi[index]['score'])

  return {label: np.max(skor) for label, skor in per_emosi.items()}

In [434]:
skor_emosi = {label: [] for label in label_emosi}

for i in range(5): # tes pakai 5 buku dulu
  isbn.append(new_books['isbn13'][i])
  kalimat = new_books['description'][i].split(".")
  prediksi = classifier(kalimat)
  max_skor = hitung_skor_emosi(prediksi)
  for label in label_emosi:
    skor_emosi[label].append(max_skor[label])

In [435]:
skor_emosi

{'anger': [np.float64(0.06413362920284271),
  np.float64(0.6126183867454529),
  np.float64(0.06413362920284271),
  np.float64(0.3514839708805084),
  np.float64(0.08141238242387772)],
 'disgust': [np.float64(0.2735919654369354),
  np.float64(0.3482854962348938),
  np.float64(0.10400665551424026),
  np.float64(0.1507224291563034),
  np.float64(0.18449531495571136)],
 'fear': [np.float64(0.9281679391860962),
  np.float64(0.9425276517868042),
  np.float64(0.9723208546638489),
  np.float64(0.36070650815963745),
  np.float64(0.0950433611869812)],
 'joy': [np.float64(0.9327974319458008),
  np.float64(0.7044206857681274),
  np.float64(0.76723712682724),
  np.float64(0.2518812119960785),
  np.float64(0.0405643954873085)],
 'sadness': [np.float64(0.6462171077728271),
  np.float64(0.8879395127296448),
  np.float64(0.5494768619537354),
  np.float64(0.7326855063438416),
  np.float64(0.8843895196914673)],
 'surprise': [np.float64(0.9671575427055359),
  np.float64(0.11169017106294632),
  np.float64(0

> Aman karena joy buku pertama ([np.float64(0.9327974319458008)) sesuai dengan yang sebelumnya

In [436]:
label_emosi = ["anger","disgust","fear","joy","sadness","surprise","neutral"]
isbn = []
skor_emosi = {label: [] for label in label_emosi}

In [437]:
for i in tqdm(range(len(new_books))): # seluruh dataset
  isbn.append(new_books['isbn13'][i])
  kalimat = new_books['description'][i].split(".")
  prediksi = classifier(kalimat)
  max_skor = hitung_skor_emosi(prediksi)
  for label in label_emosi:
    skor_emosi[label].append(max_skor[label])

100%|██████████| 5197/5197 [02:11<00:00, 39.65it/s]


In [438]:
emosi_df = pd.DataFrame(skor_emosi)
emosi_df['isbn13'] = isbn

In [439]:
emosi_df.head(3)

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273592,0.928168,0.932797,0.646217,0.967158,0.729603,9780002005883
1,0.612618,0.348285,0.942528,0.704421,0.887940,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765,9780006178736


In [440]:
emosi_books = pd.merge(new_books, emosi_df, on="isbn13")
emosi_books.head(3)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title & subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273592,0.928168,0.932797,0.646217,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612618,0.348285,0.942528,0.704421,0.887940,0.111690,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765


In [441]:
emosi_books.to_csv('emosi_books.csv',index=False)

In [442]:
from google.colab import files
files.download('emosi_books.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>